# 프로젝트: 멋진 작사가 만들기

목표 : Poetry and Lyrics가 담긴 데이터를 훈련하는 모델을 만들고, 

## Step 1. 데이터 다운로드

Song Lyrics 데이터를 다운로드 (https://www.kaggle.com/paultimothymooney/poetry/data)

## Step 2. 데이터 읽어오기와 import 정의

In [7]:
import glob                # 파일을 읽어오는 작업에 용이
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


## Step 3. 데이터 정제

문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외

#### 데이터 전처리

In [8]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>' # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("It's so good too "))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.


<start> it s so good too <end>


#### 코퍼스 생성

In [9]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence.split()) > 15: continue #문장갯수 15개를 넘으면 skip
    
    corpus.append(preprocess_sentence(sentence))
        
corpus[:2]

['<start> spoken intro <end>', '<start> you ever want something <end>']

In [13]:
len(corpus)

168357

#### 토큰화 
토큰의 max 값을 15로 지정 

In [12]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    tensor = tokenizer.texts_to_sequences(corpus)   

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2706 2456 ...    0    0    0]
 [   2    7  159 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2    7  446 ...    0    0    0]
 [   2   31  121 ...    0    0    0]
 [   2    5  351 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f054973d0d0>


In [14]:
tensor.shape

(168357, 15)

In [15]:
tensor[:1]

array([[   2, 2706, 2456,    3,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int32)

## Step 4. 평가 데이터셋 분리  

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 한다.  
단어장의 크기는 12,000 이상으로 설정, 총 데이터의 20%를 평가 데이터셋으로 사용

In [16]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2 2706 2456    3    0    0    0    0    0    0    0    0    0    0]
[2706 2456    3    0    0    0    0    0    0    0    0    0    0    0]


In [150]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.20, random_state=42)

In [151]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (134685, 14)
Target Train: (134685, 14)


## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계.  
(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출

In [152]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [154]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=30)

Epoch 1/30
526/526 [==============================] - 66s 125ms/step - loss: 3.6301
Epoch 2/30
526/526 [==============================] - 66s 126ms/step - loss: 3.1442
Epoch 3/30
526/526 [==============================] - 65s 123ms/step - loss: 2.9689
Epoch 4/30
526/526 [==============================] - 65s 124ms/step - loss: 2.8253
Epoch 5/30
526/526 [==============================] - 65s 124ms/step - loss: 2.7011
Epoch 6/30
526/526 [==============================] - 64s 121ms/step - loss: 2.5884
Epoch 7/30
526/526 [==============================] - 70s 133ms/step - loss: 2.4835
Epoch 8/30
526/526 [==============================] - 69s 132ms/step - loss: 2.3847
Epoch 9/30
526/526 [==============================] - 69s 131ms/step - loss: 2.2916
Epoch 10/30
526/526 [==============================] - 68s 129ms/step - loss: 2.2033
Epoch 11/30
526/526 [==============================] - 68s 129ms/step - loss: 2.1195
Epoch 12/30
526/526 [==============================] - 68s 130ms/step - lo

In [155]:
lyricist.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


## Step 6. 모델 평가하기
주어진 단어로 그럴듯한 문장을 만들어 내는지 확인한다.

In [156]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   
        # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [157]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '

## 느낀점

데이터 전처리 때 \<start\> it s so good too \<end\> 를 만났다.   
it s 는 it's 로 token을 만들어야할 거 같은데...  
구두점기준으로 분해하는 라이브러리도 있는거 같아 적용해 보고 싶었지만, 해보진 못했다.   
자연어처리에 관심이 생긴다면 좀 더 나은 데이터 정제를 해보고 싶다.   
흥미로운 프로젝트였다.   